# Step 1: Run NEURON simulations

We will use ICG project code to run the protocols of the Podlaski et. al (2017) on a subset of non-inactivating K-channels. 

Two subsets of such channels were hand-selected by Chaitanya Chintaluri. The selection is stored in a pickled file in the subfolder `support_files`:

In [ ]:
import pickle

kvactonly = pickle.load(open('support_files/kvact_only.pkl', 'rb'))

pow1 = kvactonly['pow1']
pow4 = kvactonly['pow4']

`pow1` contains filenames in which the gating variable appears to power 1, in `pow4` the gating variable appears to the fourth power. We will focus on the set of `pow1` channels, since iwe have more of those.

In [ ]:
print(len(pow1))
print(len(pow4))

We create subdirectories into which we will store the modfiles of the respective channels (modfiles are the NEURON models). We will download all modfiles in ICG by cloning a git repo. Then we pick out the ones from the selection:

In [ ]:
!git clone https://github.com/icgenealogy/icg-channels-K
!cd icg-channels-K && git checkout 926c5758a9ba6aec8092ac8e48d2f0c985114a37

In [ ]:
import os

!mkdir -p modfiles
!mkdir -p modfiles/pow1

# sort into subdirs
channels = os.listdir("icg-channels-K/")
for chan in channels:
    if chan in pow1:
        !cp -n -r icg-channels-K/$chan/*.mod modfiles/pow1/

There is a problem with the modfile of some channels, which we will delete:

In [ ]:
!rm modfiles/pow1/112834_kir*
!rm modfiles/pow1/150284_kir*
!rm modfiles/pow1/151458_kir*
!rm modfiles/pow1/121060_chan_KIR*

Next, we clone the ICG repos needed for NEURON simulations. 

In [ ]:
!git clone https://github.com/icgenealogy/icg-nrn-sim.git
!cd icg-nrn-sim && git checkout 4d58a1f40f779a99bd3146fda84a9acd59b07246

In [ ]:
!git clone https://github.com/icgenealogy/icg-channels-customcode.git
!cd icg-channels-customcode && git checkout 9b7b0ad0c7c18d121f2ec108a38735f4640c26d6

In [ ]:
!cp -r icg-channels-customcode/* icg-nrn-sim/icg-channels-customcode/

Setting up directories for the outputs that will be generated:

In [ ]:
!mkdir -p modfiles_results
!mkdir -p modfiles_results/pow1

We need to define the basepath of the current directory (make sure to adjust this):

In [ ]:
bp = '/home/jm/GitHub/mackelab/InferenceNeuralDynamics/fig4_channelomics/'

We are ready to run NEURON on the subset of channels. We run the modfiles for `pow1` channels only. Running NEURON for all these modfiles may take a while. It requires a functioning NEURON setup on your machine.

In [ ]:
print('{bp}/icg-nrn-sim/runbatch.pl {bp}/modfiles/pow1 {bp}/icg-nrn-sim/kvconfig.in {bp}/modfiles_results/pow1'.format(bp=bp))

In [ ]:
# run models
!$bp/icg-nrn-sim/runbatch.pl $bp/modfiles/pow1 $bp/icg-nrn-sim/kvconfig.in $bp/modfiles_results/pow1

Once the simulation have run through, we keep `modfiles_results` only.

In [ ]:
# clean up

!rm -rf icg-channels-K
!rm -rf *.dat *.txt *.hoc *.in *.mod x86_64

!rm -rf icg-nrn-sim
!rm -rf modfiles

Next, we will loop through the results files and store all traces in a pkl. We will use a special dict for it, that requires you to `pip install python-box`.

In [ ]:
from box import Box

With that, you should be able to run:

In [ ]:
import csv
import glob
import numpy as np
import os

protocols = ['ap', 'act', 'inact', 'deact', 'ramp']


# function to load tab delim data from disc
def get_matrix(file_path):
    matrix = []
    with open(file_path,'r') as f:
        reader = csv.reader(f, delimiter='\t')
        for trow in reader:
            row = []
            for tcol in trow:
                if tcol != '':
                    row.append(float(tcol))
            matrix.append(row)
    return np.asarray(matrix)

mats = Box()
for protocol in protocols:
    mats[protocol] = Box()
    mats[protocol]['data'] = []
    mats[protocol]['names'] = []
    for fpath in sorted(glob.glob(bp + '/modfiles_results/pow1/*_' + protocol + '.i')):
        mats[protocol].names.append(os.path.basename(fpath)[:-len(protocol)-3])
        mats[protocol].data.append(get_matrix(fpath))
    mats[protocol].data = np.asarray(mats[protocol].data)

In [ ]:
from support_files.pickle_macos import pickle_dump

pickle_dump(mats.to_dict(), 'support_files/pow1_mats_comp_lfs.pkl')

Number of channels simulated:

In [ ]:
len(mats.keys())

Once that executed cleanly, we can get rid of `modfiles` and `modfiles_results` if we like:

In [ ]:
!rm -rf modfiles
!rm -rf modfiles_results

Let's plot some traces:

In [ ]:
from support_files.pickle_macos import pickle_load
from box import Box

mats = Box(pickle_load('support_files/pow1_mats_comp_lfs.pkl'))
protocols = ['ap', 'act', 'inact', 'deact', 'ramp']

In [ ]:
mats['ap']['data'].shape

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

N = mats.ap.data.shape[0]

assert mats.ap.data.shape[0] == mats.act.data.shape[0] == mats.inact.data.shape[0] == mats.deact.data.shape[0] == mats.ramp.data.shape[0]
print('total N: {}'.format(N))

plot_subset = True
if plot_subset:
    N = 18
ncols = 6
nrows = N//ncols+1

for protocol in protocols:
    print(protocol)
   
    plt.figure(figsize=(19, N//ncols*3))
    for i in range(N):
        plt.subplot(nrows, ncols, i+1)
        mat = mats[protocol].data[i,:,:]
        plt.title(mats[protocol].names[i])
        plt.plot(mat[:,0], mat[:,1:]);  # first col are times, other cols are values
        plt.axis('off')
    plt.show()